<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/debugg/debugging_apis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Se entrega a mas tardar el martes 14 de Noviembre a las 11:55 pm. Vale 40 puntos de tarea.
**Te recomiendo leer todo el ejercicio antes de empezar a hacer codigo, para que te des una idea de que formato y estructura necesitamos**

In [4]:
import pandas as pd
import requests
# Completa las librerias que necesites

Objetivos del Ejercicio:

    Realizar una solicitud a la API de Pokémon para obtener datos sobre un Pokémon específico.
    Cargar y visualizar los datos en bruto para comprender su estructura.
    Limpiar y organizar los datos para un análisis posterior.
    Guardar los datos limpios en un archivo CSV.

El objetivo es crear un `DataFrame` (tidydata) de varios pokemones que ademas contengan metadata de sus moviemientos.  
Para ello deberas realizar request al `API` de pokemon, entender como vienen los datos y despues combinarlos con los `joins` correspondientes.

# API request

Crea codigo para acceder al API de pokemon y hacer requests. esta primera parte del codigo solo es para entender como es la estructura de los datos

Compelta el siguiente snippet para obter los datos del API de pokemon. La idea de estos primeros snippets es solo entender como funcionan los datos y como llegan. Para ello solo analizamos un pokemon y sus movimientos, mas adelante tenemos que repetirlo pero para varios pokemons.

## Pokemon Data

Para  el tipo de pokemon debes obtener:
1. `pokemon_id`
2. `pokemon_name`
3. `type`(s) cada pokemon tiene a 1 o dos tipos (no mas). Tienes que guardar ambos  

Guarda los `moves` del pokemon pues los usaremos mas adelante. En especial el `name` y la `url` de cada movimiento para que podamos iterar despues

In [7]:
# Elige tu pokemon
pokemon_name = 'lucario'

In [8]:
pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}'
pokemon_response = requests.get(pokemon_url)

if pokemon_response.status_code == 200:
    pokemon_data = pokemon_response.json()

    # Extract relevant information from the Pokemon data
    pokemon_moves = [move_info['move']['url'] for move_info in pokemon_data['moves']]
    pokemon_data = {
        'pokemon_id': pokemon_data['id'],
        'pokemon_name': pokemon_data['name'],
        'types': [type_info['type']['name'] for type_info in pokemon_data['types']],
        'moves': [move_info['move']['name'] for move_info in pokemon_data['moves']],
    }

    # Print the formatted Pokemon data and moves
    print(pokemon_data)
    print(pokemon_moves)
else:
    print(f"Failed to fetch data for {pokemon_name}. Status code: {pokemon_response.status_code}")

{'pokemon_id': 448, 'pokemon_name': 'lucario', 'types': ['fighting', 'steel'], 'moves': ['mega-punch', 'ice-punch', 'thunder-punch', 'swords-dance', 'mega-kick', 'headbutt', 'body-slam', 'take-down', 'roar', 'hyper-beam', 'low-kick', 'counter', 'strength', 'earthquake', 'dig', 'toxic', 'psychic', 'agility', 'quick-attack', 'screech', 'double-team', 'focus-energy', 'metronome', 'swift', 'rest', 'rock-slide', 'substitute', 'snore', 'reversal', 'protect', 'scary-face', 'mud-slap', 'foresight', 'detect', 'bone-rush', 'endure', 'swagger', 'fury-cutter', 'attract', 'sleep-talk', 'return', 'frustration', 'iron-tail', 'metal-claw', 'hidden-power', 'rain-dance', 'sunny-day', 'crunch', 'extreme-speed', 'shadow-ball', 'rock-smash', 'facade', 'focus-punch', 'helping-hand', 'role-play', 'revenge', 'brick-break', 'secret-power', 'blaze-kick', 'meteor-mash', 'rock-tomb', 'metal-sound', 'aerial-ace', 'iron-defense', 'bulk-up', 'calm-mind', 'water-pulse', 'natural-gift', 'feint', 'close-combat', 'payba

## Moves Data

Ahora para cada movimiento `move` del pokemon debes obtener los siguientes datos usando la `url`
1. `name`
2. `accuracy`
3. `type`
4. `power`
5. `pp`

In [9]:
# Solicitud para obtener datos de movimientos de un Pokémon
# Recuerda que esto es para explorar
moves = []
for m in range(len(pokemon_moves)):
    move_url = f"{pokemon_moves[m]}"
    print(move_url)
    
    move_response = requests.get(move_url)
    
    if move_response.status_code == 200:
        move_data = move_response.json()
        move_details = {
            'name': move_data['name'],
            'accuracy': move_data['accuracy'],
            'type': move_data['type']['name'],
            'power': move_data['power'],
            'pp': move_data['pp']
        }
        print(move_details)
        # Append move details to the DataFrame
        move_dataframe = pd.DataFrame(move_details, index=[0]) 
        moves.append(move_dataframe)
        print(move_dataframe)
    else:
        print(f"Failed to fetch data for move. Status code: {move_response.status_code}")

# Print the final DataFrame with move details
print(moves)

https://pokeapi.co/api/v2/move/5/
{'name': 'mega-punch', 'accuracy': 85, 'type': 'normal', 'power': 80, 'pp': 20}
         name  accuracy    type  power  pp
0  mega-punch        85  normal     80  20
https://pokeapi.co/api/v2/move/8/
{'name': 'ice-punch', 'accuracy': 100, 'type': 'ice', 'power': 75, 'pp': 15}
        name  accuracy type  power  pp
0  ice-punch       100  ice     75  15
https://pokeapi.co/api/v2/move/9/
{'name': 'thunder-punch', 'accuracy': 100, 'type': 'electric', 'power': 75, 'pp': 15}
            name  accuracy      type  power  pp
0  thunder-punch       100  electric     75  15
https://pokeapi.co/api/v2/move/14/
{'name': 'swords-dance', 'accuracy': None, 'type': 'normal', 'power': None, 'pp': 20}
           name accuracy    type power  pp
0  swords-dance     None  normal  None  20
https://pokeapi.co/api/v2/move/25/
{'name': 'mega-kick', 'accuracy': 75, 'type': 'normal', 'power': 120, 'pp': 5}
        name  accuracy    type  power  pp
0  mega-kick        75  normal  

# API Request para varios pokemons

Ahora que sabes como funciona el `API` realiza las peticiones de los siguientes pokemones respecto a sus caracteristicas
y moviemientos, y guardalas en una lista, diciconario o tabla (lo que mas te acomode). 

In [11]:
pokemons =['lucario', 'pikachu', 'gengar', 'charizard', 'blastoise', 'venusaur', 'dragonite', 'mew', 'mewtwo', 'garchomp']

In [12]:
pokemon_list = []
move_list = []
for pokemon in pokemons:
    # Datos del Pokémon
    pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{pokemon}'
    pokemon_response = requests.get(pokemon_url)

    if pokemon_response.status_code == 200:
        pokemon_data = pokemon_response.json()

        # Extract relevant information from the Pokemon data
        pokemon_moves = [move_info['move']['url'] for move_info in pokemon_data['moves']]
        pokemon_data = {
            'pokemon_id': pokemon_data['id'],
            'pokemon_name': pokemon_data['name'],
            'types': [type_info['type']['name'] for type_info in pokemon_data['types']],
            'moves': [move_info['move']['name'] for move_info in pokemon_data['moves']],
        }

        # Print the formatted Pokemon data and moves
        print(pokemon_data)
        print(pokemon_moves)
    else:
        print(f"Failed to fetch data for {pokemon}. Status code: {pokemon_response.status_code}")

    for m in range(len(pokemon_moves)):
        move_url = f"{pokemon_moves[m]}"
        print(move_url)
        
        move_response = requests.get(move_url)
        
        if move_response.status_code == 200:
            move_data = move_response.json()
            move_details = {
                'name': move_data['name'],
                'accuracy': move_data['accuracy'],
                'type': move_data['type']['name'],
                'power': move_data['power'],
                'pp': move_data['pp']
            }
            # Append move details to the DataFrame
            move_list.append(move_details)
            print(move_details)
        else:
            print(f"Failed to fetch data for move. Status code: {move_response.status_code}")

    pokemon_list.append(pokemon_data)

print(pokemon_list)
print(move_list)

{'pokemon_id': 448, 'pokemon_name': 'lucario', 'types': ['fighting', 'steel'], 'moves': ['mega-punch', 'ice-punch', 'thunder-punch', 'swords-dance', 'mega-kick', 'headbutt', 'body-slam', 'take-down', 'roar', 'hyper-beam', 'low-kick', 'counter', 'strength', 'earthquake', 'dig', 'toxic', 'psychic', 'agility', 'quick-attack', 'screech', 'double-team', 'focus-energy', 'metronome', 'swift', 'rest', 'rock-slide', 'substitute', 'snore', 'reversal', 'protect', 'scary-face', 'mud-slap', 'foresight', 'detect', 'bone-rush', 'endure', 'swagger', 'fury-cutter', 'attract', 'sleep-talk', 'return', 'frustration', 'iron-tail', 'metal-claw', 'hidden-power', 'rain-dance', 'sunny-day', 'crunch', 'extreme-speed', 'shadow-ball', 'rock-smash', 'facade', 'focus-punch', 'helping-hand', 'role-play', 'revenge', 'brick-break', 'secret-power', 'blaze-kick', 'meteor-mash', 'rock-tomb', 'metal-sound', 'aerial-ace', 'iron-defense', 'bulk-up', 'calm-mind', 'water-pulse', 'natural-gift', 'feint', 'close-combat', 'payba

# Evolucion a  Tabular

Ahora para cada request (`pokemon` y `movimiento`) convierte su informacion a datos tabulares. Toma en cuenta que vamos a realizar operaciones sobre esos DataFrames, por lo tanto tienes que elegir un formato adecuado.  
Para cada pokemon, y para cada movimiento debe existir un `DataFrame` o puedes guardarlos en uno solo, aunque es recomendable dejarlos por separado.

In [13]:
pokemon_df = pd.DataFrame(pokemon_list)
pokemon_tipos = pokemon_df.explode('types')
move_df = pd.DataFrame(move_list)
move_df = move_df.dropna()

print(pokemon_df)
print(pokemon_tipos)
print('\nBase de movimientos\n')
print(move_df)

   pokemon_id pokemon_name              types  \
0         448      lucario  [fighting, steel]   
1          25      pikachu         [electric]   
2          94       gengar    [ghost, poison]   
3           6    charizard     [fire, flying]   
4           9    blastoise            [water]   
5           3     venusaur    [grass, poison]   
6         149    dragonite   [dragon, flying]   
7         151          mew          [psychic]   
8         150       mewtwo          [psychic]   
9         445     garchomp   [dragon, ground]   

                                               moves  
0  [mega-punch, ice-punch, thunder-punch, swords-...  
1  [mega-punch, pay-day, thunder-punch, slam, dou...  
2  [mega-punch, fire-punch, ice-punch, thunder-pu...  
3  [mega-punch, fire-punch, thunder-punch, scratc...  
4  [mega-punch, ice-punch, mega-kick, headbutt, t...  
5  [swords-dance, cut, bind, vine-whip, headbutt,...  
6  [mega-punch, fire-punch, ice-punch, thunder-pu...  
7  [pound, mega-punc

# Union de Informacion

1. Obten el nombre y caracteristicas de todos lo movimientos que aparecen en por lo menos 3 pokemons de los de la lista (osea por lo menos 3 pokemons pueden realizar el movimiento), y guarda los datos de movimiento en un solo `DataFrame`

In [14]:
grouped_moves = move_df.copy()
grouped_moves['count'] = grouped_moves.groupby('name')['name'].transform('count')
print(grouped_moves)

repetiDF = grouped_moves[grouped_moves['count'] >= 3]
repetiDF = repetiDF.set_index('name').drop_duplicates()
print(repetiDF)


                 name  accuracy      type  power  pp  count
0          mega-punch      85.0    normal   80.0  20      8
1           ice-punch     100.0       ice   75.0  15      6
2       thunder-punch     100.0  electric   75.0  15      7
4           mega-kick      75.0    normal  120.0   5      8
5            headbutt     100.0    normal   70.0  15     10
...               ...       ...       ...    ...  ..    ...
1408      liquidation     100.0     water   85.0  10      3
1409   breaking-swipe     100.0    dragon   60.0  15      4
1410       scale-shot      90.0    dragon   25.0  20      4
1411  scorching-sands     100.0    ground   70.0  10      3
1412       tera-blast     100.0    normal   80.0  10      8

[836 rows x 6 columns]
                  accuracy      type  power  pp  count
name                                                  
mega-punch            85.0    normal   80.0  20      8
ice-punch            100.0       ice   75.0  15      6
thunder-punch        100.0  electric

1.1 Salva los datos en un archivo csv

In [15]:
pd.DataFrame().to_csv('pokemon_moves_3.csv',index=False)

2. Obten el nombre y caracteristicas todos los moviemientos que no se repiten en ningun otro pokemon, osea son movimientos unicos para un solo pokemon

In [16]:
sinRepetiDF = grouped_moves[grouped_moves['count'] == 1]
sinRepetiDF = sinRepetiDF.set_index('name').drop_duplicates()
print(sinRepetiDF)

                  accuracy      type  power  pp  count
name                                                  
bone-rush             90.0    ground   25.0  10      1
meteor-mash           90.0     steel   90.0  10      1
force-palm           100.0  fighting   60.0  10      1
double-kick          100.0  fighting   30.0  30      1
thunder-shock        100.0  electric   40.0  30      1
...                    ...       ...    ...  ..    ...
meteor-beam           90.0      rock  120.0  10      1
misty-explosion      100.0     fairy  100.0   5      1
burning-jealousy     100.0      fire   70.0   5      1
triple-axel           90.0       ice   20.0  10      1
psystrike            100.0   psychic  100.0  10      1

[62 rows x 5 columns]


2.2 Salvalos a un csv

In [17]:
pd.DataFrame().to_csv('unrepeated_moves.csv',index=False)

# Manejo de tipos

Para cada tipo de pokemon (fire, water, grass, etc) que aparecen en los datos. Obten una serie de dataframes que contengan a los pokemones que son de cada tipo. Y colocalos en un diccionarion donde la llave (`key`) es el tipo

In [18]:
pokemon_dict = {}


for tipo, grupo in pokemon_tipos.groupby('types'):
    
    pokemon_dict[tipo] = grupo


print(pokemon_dict)

{'dragon':    pokemon_id pokemon_name   types  \
6         149    dragonite  dragon   
9         445     garchomp  dragon   

                                               moves  
6  [mega-punch, fire-punch, ice-punch, thunder-pu...  
9  [swords-dance, cut, sand-attack, headbutt, tac...  , 'electric':    pokemon_id pokemon_name     types  \
1          25      pikachu  electric   

                                               moves  
1  [mega-punch, pay-day, thunder-punch, slam, dou...  , 'fighting':    pokemon_id pokemon_name     types  \
0         448      lucario  fighting   

                                               moves  
0  [mega-punch, ice-punch, thunder-punch, swords-...  , 'fire':    pokemon_id pokemon_name types  \
3           6    charizard  fire   

                                               moves  
3  [mega-punch, fire-punch, thunder-punch, scratc...  , 'flying':    pokemon_id pokemon_name   types  \
3           6    charizard  flying   
6         149    drago

Despues guara ese diccionario en un json llamado `tipos.json`

In [19]:
import json

tipos_json = {}
for tipo, dataframe in pokemon_dict.items():
    tipos_json[tipo] = json.loads(dataframe.to_json(orient='records'))
print(tipos_json)

with open('tipos.json', 'w') as json_file:
    json.dump(tipos_json, json_file)

print("Dictionary saved to tipos.json")

{'dragon': [{'pokemon_id': 149, 'pokemon_name': 'dragonite', 'types': 'dragon', 'moves': ['mega-punch', 'fire-punch', 'ice-punch', 'thunder-punch', 'razor-wind', 'cut', 'wing-attack', 'fly', 'bind', 'slam', 'mega-kick', 'headbutt', 'horn-drill', 'body-slam', 'wrap', 'take-down', 'double-edge', 'leer', 'roar', 'flamethrower', 'mist', 'water-gun', 'hydro-pump', 'surf', 'ice-beam', 'blizzard', 'bubble-beam', 'hyper-beam', 'low-kick', 'strength', 'dragon-rage', 'fire-spin', 'thunderbolt', 'thunder-wave', 'thunder', 'earthquake', 'toxic', 'agility', 'rage', 'mimic', 'double-team', 'light-screen', 'haze', 'reflect', 'bide', 'metronome', 'fire-blast', 'waterfall', 'swift', 'skull-bash', 'rest', 'rock-slide', 'substitute', 'snore', 'curse', 'protect', 'scary-face', 'mud-slap', 'zap-cannon', 'icy-wind', 'detect', 'outrage', 'sandstorm', 'endure', 'swagger', 'fury-cutter', 'steel-wing', 'attract', 'sleep-talk', 'return', 'frustration', 'safeguard', 'dynamic-punch', 'dragon-breath', 'encore', 'ir

# Seleccion de Pokemones

Ahora crea un DataFrame que para cada pokemon unicamente contenga los movimientos que son de su tipo (osea si es tipo fuego y voldaor solo usa movimientos de tipo volador y fuego). Y crea una tabla donde tengan los siguientes datos en formato `tidydata`:
1. `pokemon_id`
2. `pokemon_name`
3. `type`(s) cada pokemon tiene a 1 o dos tipos (no mas). Tienes que guardar ambos  
4. `name` (del movimiento)
5. `accuracy`
6. `type`
7. `power`
8. `pp`

In [20]:
pokemon_tempo = pokemon_df.copy()
pokemon_tempo = pokemon_tempo.explode('moves')
print(pokemon_tempo)

moves_unique = move_df.drop_duplicates()
moves_unique = moves_unique.rename(columns={'name':'moves'})
print(moves_unique)

pokemoves_temp = pd.merge(pokemon_tempo, moves_unique, on='moves', how='inner')
print(pokemoves_temp)

pokemoves = pokemoves_temp[pokemoves_temp.apply(lambda row: row['type'] in row['types'], axis=1)]
print(pokemoves)


    pokemon_id pokemon_name              types            moves
0          448      lucario  [fighting, steel]       mega-punch
0          448      lucario  [fighting, steel]        ice-punch
0          448      lucario  [fighting, steel]    thunder-punch
0          448      lucario  [fighting, steel]     swords-dance
0          448      lucario  [fighting, steel]        mega-kick
..         ...          ...                ...              ...
9          445     garchomp   [dragon, ground]      liquidation
9          445     garchomp   [dragon, ground]   breaking-swipe
9          445     garchomp   [dragon, ground]       scale-shot
9          445     garchomp   [dragon, ground]  scorching-sands
9          445     garchomp   [dragon, ground]       tera-blast

[1413 rows x 4 columns]
                 moves  accuracy      type  power  pp
0           mega-punch      85.0    normal   80.0  20
1            ice-punch     100.0       ice   75.0  15
2        thunder-punch     100.0  electric   

Finalmente guarda los datos en `.parquet`

In [23]:
pokemoves.to_parquet('pokemoves.parquet', index=False)